In [13]:
# Imports
import os
import glob
import pandas as pd

from contextlib import contextmanager
from time import time

In [14]:
# Constants
DATA_DIR = "/home/sagemaker-user/LSTM Project/CS7643ProjectData/"

In [15]:
@contextmanager
def timer(name):
    start = time()
    yield
    end = time()
    print(f"{name}: {end - start:.2f} seconds")

In [16]:
# Read options prices into a dataframe
def read_dataset(directory: str, file_pattern: str, label: str) -> pd.DataFrame:
    path_pattern = os.path.join(directory, file_pattern)
    options_files = glob.glob(path_pattern)

    dfs = []
    with timer(f"{label} file reading"):
        for filename in options_files:
            try:
                df = pd.read_csv(filename, low_memory=False)
                dfs.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {str(e)}")
                continue

        if not dfs:
            raise ValueError("No files were successfully read")

        combined_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    return combined_df

In [17]:
def _clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.map(lambda x: x.strip('[] '))
    return df


def _drop_columns(df: pd.DataFrame, drop_columns: list) -> pd.DataFrame:
    for column in drop_columns:
        try:
            df = df.drop(column, axis=1)
        except KeyError as e:
            print(f"! Warning KeyError: {str(e)}")
    return df


def _drop_long_options(df: pd.DataFrame, expiry_in_years: int = 3, print_distribution: bool = True) -> (pd.DataFrame, int):
    # Calculate years to maturity
    df['YEARS_TO_MATURITY'] = (df['EXPIRE_DATE'] - df['QUOTE_DATE']).dt.total_seconds() / (365.25 * 24 * 60 * 60)

    if print_distribution:
        # Count records by year ranges
        year_ranges = pd.cut(df['YEARS_TO_MATURITY'],
                             bins=[-float('inf'), 1, 2, 3, float('inf')],
                             labels=['0-1 years', '1-2 years', '2-3 years', '3+ years'])

        print("\nDistribution of maturity periods:")
        distribution = year_ranges.value_counts().sort_index()
        for period, count in distribution.items():
            print(f"{period}: {count} records ({count/len(df)*100:.2f}%)")

    # Filter records
    df = df[df['YEARS_TO_MATURITY'] < expiry_in_years]
    return df, distribution['3+ years']


def _print_preprocessing_stats(df: pd.DataFrame, starting_row_count: int, nan_row_count: int, filtered_count: int):
    final_row_count = starting_row_count - nan_row_count - filtered_count

    print(df.columns)
    print(f"Original row count: {starting_row_count}")
    print(f"NaN row count: {nan_row_count} records ({nan_row_count/starting_row_count*100:.2f}%)")
    print(f"Filtered record count: {filtered_count} records ({filtered_count/starting_row_count*100:.2f}%)")
    print(f"Post-processed row counts: {final_row_count} records ({final_row_count/starting_row_count*100:.2f}%)")


# Preprocess options dataframe
def preprocess_options_df(df: pd.DataFrame) -> pd.DataFrame:
    with timer("Preprocessing Options Dataset"):
        initial_row_count = len(df)

        # Remove square brackets and strip whitespace from column names
        df = _clean_column_names(df)

        # Clean options dataframe
        nan_counts = df.isna().any(axis=1).sum()
        df = df.dropna()

        # Format DateTime columns
        df['QUOTE_DATE'] = pd.to_datetime(df['QUOTE_DATE'])
        df['EXPIRE_DATE'] = pd.to_datetime(df['EXPIRE_DATE'])

        # Sort by QUOTE_DATE
        df = df.sort_values('QUOTE_DATE').reset_index(drop=True)

        # Drop options expiring in 3 years
        df, filtered_count = _drop_long_options(df)

        # Remove redundant or unecessary columns
        drop_columns = ["QUOTE_UNIXTIME", "QUOTE_READTIME", "QUOTE_TIME_HOURS",
                        "EXPIRE_UNIX", "YEARS_TO_MATURITY"]
        df = _drop_columns(df, drop_columns)

        _print_preprocessing_stats(df, initial_row_count, nan_counts, filtered_count)
        return df

In [20]:
def preprocess_rates_df(df):
    with timer("Preprocessing Rates Dataset"):
        initial_rows = len(df)
        df['Date'] = pd.to_datetime(df['Date'])

        df = df.sort_values('Date').reset_index(drop=True)

        cutoff = pd.to_datetime('2022-04-01')
        df_filtered = df[df['Date'] >= cutoff]
        removed_rows = initial_rows - len(df_filtered)

        # Print summary
        print(f"\nDate Filtering Summary:")
        print(f"Cutoff date: {cutoff.strftime('%Y-%m-%d')}")
        print(f"Original records: {initial_rows:,}")
        print(f"Records removed: {removed_rows:,}")
        print(f"Records remaining: {len(df_filtered):,}")
        print(f"Percentage removed: {(removed_rows/initial_rows)*100:.2f}%")

        return df_filtered

In [23]:
options_df = read_dataset(DATA_DIR, "*.txt", "Options Data")
options_df = preprocess_options_df(options_df)

rates_df = read_dataset(DATA_DIR, "*.csv", "Rates Data")
rates_df = preprocess_rates_df(rates_df)

df = pd.merge(
    options_df,
    rates_df,
    left_on='QUOTE_DATE',
    right_on='Date',
    how='left'
)

df = _drop_columns(df, ["Date"])
df.to_csv(f"{DATA_DIR}/data.csv")
df

,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2022-04-01,4545.81,2022-04-01,0.0,1.0,0.0,0.0,-0.01152,0.016,,...,NaN,1.09,1.72,2.44,2.61,2.55,2.50,2.39,2.6,2.44
1,2022-04-01,4545.81,2022-07-15,105.0,0.01341,0.0001,0.84155,-0.05319,0.16936,0.138600,...,NaN,1.09,1.72,2.44,2.61,2.55,2.50,2.39,2.6,2.44
2,2022-04-01,4545.81,2022-07-15,105.0,0.01449,0.00007,0.93935,-0.05841,0.19242,0.138330,...,NaN,1.09,1.72,2.44,2.61,2.55,2.50,2.39,2.6,2.44
3,2022-04-01,4545.81,2022-07-15,105.0,0.0168,0.00011,1.02739,-0.06405,0.21393,0.136820,...,NaN,1.09,1.72,2.44,2.61,2.55,2.50,2.39,2.6,2.44
4,2022-04-01,4545.81,2022-07-15,105.0,0.01812,0.00014,1.13223,-0.07021,0.23942,0.136170,...,NaN,1.09,1.72,2.44,2.61,2.55,2.50,2.39,2.6,2.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507157,2023-12-29,4772.17,2024-01-24,26.0,0.75734,0.00208,4.03642,-1.29222,2.55497,0.117180,...,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.2,4.03
3507158,2023-12-29,4772.17,2024-01-24,26.0,0.75201,0.00215,4.08014,-1.27471,2.54229,0.113690,...,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.2,4.03
3507159,2023-12-29,4772.17,2024-01-24,26.0,0.73612,0.00217,4.21449,-1.31733,2.48788,0.115890,...,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.2,4.03
3507160,2023-12-29,4772.17,2024-01-24,26.0,0.79279,0.00185,3.68749,-1.24747,2.66849,0.118640,...,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.2,4.03
